## Imports and config

In [ ]:
#Append other folders to path
import sys
sys.path.append('../')

#Automatically reload modules
%load_ext autoreload
%autoreload 2

#Make this notebook full width
from IPython.core.display import display, HTML
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

#Imports
import numpy as np
import keras,os
from Utils.zscore_images import *
from IO.load_training_data import *
from Models.starter_CNN import *

np.random.seed(42) #set numpy seed to easy reproducibility

training_portion = 0.8 # use 80% of samples for training, rest validation

## Load data

In [ ]:
X,y = load_training_data() # Loads the training data from /Data/

split_idx = int(np.round(training_portion * y.shape[0])) #Compute index where we split training/validation

## Setup model

In [ ]:
model = starter_CNN() # Sets up a simple CNN

In [ ]:
#Setup keras training
batch_size = 50
model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(0.01),
              metrics=['accuracy'])

##  Run a training epoch

In [ ]:
iterations = np.floor(split_idx / batch_size).astype(int) #this discards a few images due to below iterationcount floor, should be fixed at some point
loss,acc = 0,0 #running loss and accuracy
with trange(iterations) as t:
    for i in t:
        start_idx = i * batch_size 
        x_batch = zscore_images(X[start_idx:start_idx+batch_size])
        y_batch = y[start_idx:start_idx+batch_size]
        
        #run batch
        metrics = model.train_on_batch(x_batch, y_batch)
        
        #print metrics
        loss = loss + metrics[0] 
        acc = acc + metrics[1]
        t.set_description('Running training')
        t.set_postfix(loss="%.2f" % round(loss / (i+1),2),acc="%.2f" % round(acc / (i+1),2))

## Run a validation epoch

In [ ]:
iterations = np.floor((y.shape[0]-split_idx) / batch_size).astype(int) #this discards a few images due to below iterationcount floor, should be fixed at some point
loss,acc = 0,0 #running loss and accuracy
with trange(iterations) as t:
    for i in t:
        #create batch
        start_idx = split_idx + i * batch_size 
        x_batch = zscore_images(X[start_idx:start_idx+batch_size])
        y_batch = y[start_idx:start_idx+batch_size]
        
        #run batch
        metrics = model.test_on_batch(x_batch, y_batch)
        
        #print metrics
        loss = loss + metrics[0] 
        acc = acc + metrics[1]
        t.set_description('Running validation')
        t.set_postfix(loss="%.2f" % round(loss / (i+1),2),acc="%.2f" % round(acc / (i+1),2))
        
print("Validation loss:",loss / iterations)
print("Validation accuracy:",acc / iterations)